[**Blueprints for Text Analysis Using Python**](https://github.com/blueprints-for-text-analytics-python/blueprints-text)  
Jens Albrecht, Sidharth Ramachandran, Christian Winkler

# Chapter 4:<div class='tocSkip'/>

# How to Prepare Textual Data For Statistics and Machine Learning

## Remark<div class='tocSkip'/>

The code in this notebook differs slightly from the printed book. For example we frequently use pretty print (`pp.pprint`) instead of `print` and `tqdm`'s `progress_apply` instead of Pandas' `apply`. 

Moreover, several layout and formatting commands, like `figsize` to control figure size or subplot commands are removed in the book.

You may also find some lines marked with three hashes ###. Those are not in the book as well as they don't contribute to the concept.

All of this is done to simplify the code in the book and put the focus on the important parts instead of formatting.

## Setup<div class='tocSkip'/>

Set directory locations. If working on Google Colab: copy files and install required libraries.

In [ ]:
import sys, os
ON_COLAB = 'google.colab' in sys.modules

if ON_COLAB:
    GIT_ROOT = 'https://github.com/blueprints-for-text-analytics-python/blueprints-text/raw/master'
    os.system(f'wget {GIT_ROOT}/ch04/setup.py')

%run -i setup.py

## Load Python Settings<div class="tocSkip"/>

Common imports, defaults for formatting in Matplotlib, Pandas etc.

In [ ]:
%run "$BASE_DIR/settings.py"

%reload_ext autoreload
%autoreload 2
%config InlineBackend.figure_format = 'png'

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# otherwise text between $ signs will be interpreted as formula and printed in italic
pd.set_option('display.html.use_mathjax', False)
sys.path.append(BASE_DIR + '/packages')

# How to Prepare Textual Data For Statistics and Machine Learning
## What you'll learn and what we build


# A Data Preprocessing Pipeline


# Introducing the Data Set: Reddit Self Posts


## Loading Data into Pandas


In [ ]:
posts_file = f"{BASE_DIR}/data/reddit-selfposts/rspct.tsv.gz"
posts_file = f"{BASE_DIR}/data/reddit-selfposts/rspct_autos.tsv.gz" ### for faster loads use this subset
posts_df = pd.read_csv(posts_file, sep='\t')

subred_file = f"{BASE_DIR}/data/reddit-selfposts/subreddit_info.csv.gz"
subred_df = pd.read_csv(subred_file).set_index(['subreddit'])

df = posts_df.join(subred_df, on='subreddit')
len(df) ###

## Standardizing Attribute Names


In [ ]:
print(df.columns)

In [ ]:
column_mapping = {
    'id': 'id',
    'subreddit': 'subreddit',
    'title': 'title',
    'selftext': 'text',
    'category_1': 'category',
    'category_2': 'subcategory',  
    'category_3': None, # no data
    'in_data': None, # not needed
    'reason_for_exclusion': None # not needed
}

# define remaining columns
columns = [c for c in column_mapping.keys() if column_mapping[c] != None]

# select and rename those columns
df = df[columns].rename(columns=column_mapping)

In [ ]:
df = df[df['category'] == 'autos']
len(df) ###

In [ ]:
pd.options.display.max_colwidth = None ###
df.sample(1, random_state=7).T
pd.options.display.max_colwidth = 200 ###

## Checking for Missing Values


In [ ]:
df.isna().sum()

## Saving and Loading a Data Frame


In [ ]:
df.to_pickle("reddit_dataframe.pkl")

In [ ]:
import sqlite3

db_path = f"{BASE_DIR}/data/reddit-selfposts/reddit-selfposts.db"

con = sqlite3.connect(db_path)
df.to_sql("posts", con, index=False, if_exists="replace")
con.close()

In [ ]:
import sqlite3 ###
db_path = f"{BASE_DIR}/data/reddit-selfposts/reddit-selfposts.db" ###
con = sqlite3.connect(db_path)
df = pd.read_sql("select * from posts", con)
con.close()

In [ ]:
len(df)

# Cleaning Textual Data with Regular Expressions


In [ ]:
text = """
After viewing the [PINKIEPOOL Trailer](https://www.youtu.be/watch?v=ieHRoHUg)
it got me thinking about the best match ups.
<lb>Here's my take:<lb><lb>[](/sp)[](/ppseesyou) Deadpool<lb>[](/sp)[](/ajsly)
Captain America<lb>"""

text = text.replace('\n', ' ').strip() ###
print(text) ###

## Blueprint: Identifying Dirty Data


In [ ]:
import re

RE_SUSPICIOUS = re.compile(r'[&#<>{}\[\]\\]')

def impurity(text, min_len=10):
    if text == None or len(text) < min_len:
        return 0
    else:
        # return share of suspicious characters in a text
        return len(RE_SUSPICIOUS.findall(text))/len(text)

print(impurity(text))

In [ ]:
pd.options.display.max_colwidth = 100 ###
# add new column to data frame
df['impurity'] = df['text'].progress_apply(impurity, min_len=10)

# get the top 3 records
df[['text', 'impurity']].sort_values(by='impurity', ascending=False).head(3)
pd.options.display.max_colwidth = 200 ###

In [ ]:
from blueprints.exploration import count_words
count_words(df, column='text', preprocess=lambda t: re.findall(r'<[\w/]*>', t))

## Blueprint: Text-Cleaning with Regular Expressions


In [ ]:
import html

def clean(text):
    # convert html escapes like &amp; to characters.
    text = html.unescape(text) 
    # tags like <tab>
    text = re.sub(r'<[^<>]*>', ' ', text)
    # markdown URLs like [Some text](https://....)
    text = re.sub(r'\[([^\[\]]*)\]\([^\(\)]*\)', r'\1', text)
    # text or code in brackets like [0]
    text = re.sub(r'\[[^\[\]]*\]', ' ', text)
    # standalone sequences of specials, matches &# but not #cool
    text = re.sub(r'(?:^|\s)[&#<>{}\[\]+|\\:-]{1,}(?:\s|$)', ' ', text)
    # standalone sequences of hyphens like --- or ==
    text = re.sub(r'(?:^|\s)[\-=\+]{2,}(?:\s|$)', ' ', text)
    # sequences of white spaces
    text = re.sub(r'\s+', ' ', text)
    return text.strip()

In [ ]:
clean_text = clean(text)
print(clean_text)
print("Impurity:", impurity(clean_text))

In [ ]:
df['clean_text'] = df['text'].progress_apply(clean)
df['impurity']   = df['clean_text'].apply(impurity, min_len=20)

In [ ]:
df[['clean_text', 'impurity']].sort_values(by='impurity', ascending=False).head(3)

## Removing Noise with textacy 


In [ ]:
from textacy.preprocessing.resources import RE_URL

count_words(df, column='clean_text', preprocess=RE_URL.findall).head(3)

### Pattern-based Data Masking with Textacy


In [ ]:
from textacy.preprocessing.replace import replace_urls

text = "Check out https://spacy.io/usage/spacy-101"

# using default substitution _URL_
print(replace_urls(text))

### Unicode Character Normalization


In [ ]:
text = "The café “Saint-Raphaël” is loca-\nted on Côte dʼAzur."

In [ ]:
import textacy.preprocessing as tprep

def normalize(text):
    text = tprep.normalize_hyphenated_words(text)
    text = tprep.normalize_quotation_marks(text)
    text = tprep.normalize_unicode(text)
    text = tprep.remove_accents(text)
    return text

print(normalize(text))

In [ ]:
df['clean_text'] = df['clean_text'].progress_map(normalize)

In [ ]:
df['text'] = df['clean_text']
df.drop(columns=['clean_text', 'impurity'], inplace=True)

db_path = f"{BASE_DIR}/data/reddit-selfposts/reddit-selfposts.db" ###
con = sqlite3.connect(db_path)
df.to_sql("posts_cleaned", con, index=False, if_exists="replace")
con.close()

# Tokenization


In [ ]:
text = """
2019-08-10 23:32: @pete/@louis - I don't have a well-designed 
solution for today's problem. The code of module AC68 should be -1. 
Have to think a bit... #goodnight ;-) 😩😬"""

## Tokenization with Regular Expressions


In [ ]:
tokens = re.findall(r'\w\w+', text)
print("|".join(tokens))

In [ ]:
RE_TOKEN = re.compile(r"""
               ( [#]?[@\w'’\.\-\:]*\w     # words, hash tags and email adresses
               | [:;<]\-?[\)\(3]          # coarse pattern for basic text emojis
               | [\U0001F100-\U0001FFFF]  # coarse code range for unicode emojis
               )
               """, re.VERBOSE)

def tokenize(text):
    return RE_TOKEN.findall(text)

tokens = tokenize(text)
print("|".join(tokens))

In [ ]:
df['tokens'] = df['text'].progress_map(tokenize)

## Tokenization with NLTK


In [ ]:
import nltk

nltk.download('punkt') ###
tokens = nltk.tokenize.word_tokenize(text)
print("|".join(t for t in tokens))

## Recommendations for Tokenization


# Linguistic Processing with spaCy


## Instantiating a Pipeline


In [ ]:
import spacy
nlp = spacy.load('en_core_web_sm')

In [ ]:
nlp.pipeline

In [ ]:
nlp = spacy.load("en", disable=["parser", "ner"])

## Processing Text


In [ ]:
nlp = spacy.load("en")
text = "My best friend Ryan Peters likes fancy adventure games."
doc = nlp(text)

In [ ]:
for token in doc:
    print(token, end="|")

In [ ]:
def display_nlp(doc, include_punct=False):
    """Generate data frame for visualization of spaCy tokens."""
    rows = []
    for i, t in enumerate(doc):
        if not t.is_punct or include_punct:
            row = {'token': i,  'text': t.text, 'lemma_': t.lemma_, 
                   'is_stop': t.is_stop, 'is_alpha': t.is_alpha,
                   'pos_': t.pos_, 'dep_': t.dep_, 'ent_type_': t.ent_type_}
            rows.append(row)
    
    df = pd.DataFrame(rows).set_index('token')
    df.index.name = None
    return df

## Customizing Tokenization


In [ ]:
text = "@Pete: choose low-carb #food #eat-smart. _url_ ;-) 😋👍"
nlp = spacy.load('en') ###
doc = nlp(text)

print(*[token for token in doc], sep="|")

In [ ]:
import re ###
import spacy ###
from spacy.tokenizer import Tokenizer
from spacy.util import compile_prefix_regex, \
                       compile_infix_regex, compile_suffix_regex

def custom_tokenizer(nlp):
    
    # use default patterns except the ones matched by re.search
    prefixes = [pattern for pattern in nlp.Defaults.prefixes 
                if pattern not in ['-', '_', '#']]
    suffixes = [pattern for pattern in nlp.Defaults.suffixes
                if pattern not in ['_']]
    infixes  = [pattern for pattern in nlp.Defaults.infixes
                if not re.search(pattern, 'xx-xx')]

    return Tokenizer(vocab          = nlp.vocab, 
                     rules          = nlp.Defaults.tokenizer_exceptions,
                     prefix_search  = compile_prefix_regex(prefixes).search,
                     suffix_search  = compile_suffix_regex(suffixes).search,
                     infix_finditer = compile_infix_regex(infixes).finditer,
                     token_match    = nlp.Defaults.token_match)

nlp = spacy.load('en')
nlp.tokenizer = custom_tokenizer(nlp)

doc = nlp(text)
print(*[token for token in doc], sep="|")

## Lemmatization


## Stop Word Detection


In [ ]:
from spacy.lang.en import STOP_WORDS as stop_words
print(len(stop_words))

In [ ]:
nlp = spacy.load('en')
nlp.vocab['down'].is_stop = False
nlp.vocab['Dear'].is_stop = True
nlp.vocab['Regards'].is_stop = True

## Part-of-Speech Tagging


## ADD? Dependency Parsing and Named Entity Recognition


In [ ]:
text = "My friend Ryan Peters doesn't like Florida."

In [ ]:
from spacy import displacy

nlp = spacy.load('en') ###
doc = nlp(text)
displacy.render(doc, style='dep', options={'compact': False, 'distance': 100})

## Named-Entity Recognition


In [ ]:
from spacy import displacy ###

text = "James O'Neill, chairman of World Cargo Inc, lives in San Francisco."
doc = nlp(text)

displacy.render(doc, style='ent', jupyter=True)

# Blueprints for Feature Extraction


## Extracting Words based on Part-of-Speech


In [ ]:
text = "My best friend Ryan Peters likes fancy adventure games."
doc = nlp(text)

nouns = [t for t in doc if t.pos_ in ['NOUN', 'PROPN']]
print(nouns)

In [ ]:
import textacy

text = "My best friend Ryan Peters likes fancy adventure games."
doc = nlp(text)

tokens = textacy.extract.words(doc, 
            filter_stops = True,           # default True, no stopwords
            filter_punct = True,           # default True, no punctuation
            filter_nums = True,            # default False, no numbers
            include_pos = ['NOUN', 'PROPN'], # default None = include all
            exclude_pos = None,            # default None = exclude none
            min_freq = 1)                  # minimum frequency of words

In [ ]:
def extract_lemmas(doc, **kwargs):
    return [t.lemma_ for t in textacy.extract.words(doc, **kwargs)]

lemmas = extract_lemmas(doc, include_pos=['ADJ', 'NOUN'])
print(*lemmas, sep='|')

## Extracting Noun Chunks


In [ ]:
spans = textacy.extract.matches(doc, patterns=["POS:ADJ:? POS:NOUN:+"])
print(*spans, sep='|')

In [ ]:
print(*doc.noun_chunks, sep='|')

In [ ]:
def extract_noun_chunks(doc, include_pos=['NOUN'], sep='_'):
    chunks = []
    for noun_chunk in doc.noun_chunks:
        chunk = [token.lemma_ for token in noun_chunk
                 if token.pos_ in include_pos]
        if len(chunk) >= 2:
            chunks.append(sep.join(chunk))
    return chunks

In [ ]:
noun_chunks = extract_noun_chunks(doc, include_pos=['ADJ', 'NOUN', 'PROPN'])
print(*noun_chunks, sep='|')

## Extracting Named Entities


In [ ]:
def extract_entities(doc, include_types=None, sep='_'):

    ents = textacy.extract.entities(doc, 
             include_types=include_types, 
             exclude_types=None, 
             drop_determiners=True, 
             min_freq=1)
    
    return [re.sub('\s+', sep, e.lemma_)+'/'+e.label_ for e in ents]

In [ ]:
nlp = spacy.load('en') ###
text = "George Washington was the first president of the United States."
doc = nlp(text)

entities = extract_entities(doc, ['PERSON', 'GPE'])
print(*entities, sep='|')

# Extracting NLP Features on a Large Dataset
## One Function to Get It All


In [ ]:
nlp = spacy.load('en') # load model
nlp.tokenizer = custom_tokenizer(nlp) # optional

def extract_nlp(doc):

    # doc = nlp(text)
    
    lemmas          = extract_lemmas(doc, exclude_pos = ['PART', 'PUNCT', 
                                           'DET', 'PRON', 'SYM', 'SPACE'],
                                          filter_stops = False)
    adjs_verbs      = extract_lemmas(doc, include_pos = ['ADJ', 'VERB'])
    nouns           = extract_lemmas(doc, include_pos = ['NOUN', 'PROPN'])
    noun_chunks     = extract_noun_chunks(doc, ['NOUN'])
    adj_noun_chunks = extract_noun_chunks(doc, ['NOUN', 'ADJ'])
    entities        = extract_entities(doc, ['PERSON', 'ORG', 'GPE', 'LOC'])

    return lemmas, adjs_verbs, nouns, noun_chunks, adj_noun_chunks, entities

In [ ]:
text = "My best friend Ryan Peters likes fancy adventure games."
doc = nlp(text)
print(*extract_nlp(doc), sep='\n')

## Creating Multiple Columns in a Data Frame


In [ ]:
import sqlite3 ###
db_path = f"{BASE_DIR}/data/reddit-selfposts/reddit-selfposts.db" ###
con = sqlite3.connect(db_path)
df = pd.read_sql("select * from posts_cleaned", con)
con.close()

df['text'] = df['title'] + ': ' + df['text']

In [ ]:
# define column names
nlp_columns = ['lemmas', 'adjs_verbs', 'nouns', 'noun_chunks', 
               'adj_noun_chunks', 'entities']

df[nlp_columns] = df.progress_apply(lambda row: nlp_extract(row['text']), 
                                    axis='columns', result_type='expand')

In [ ]:
count_words(df, 'noun_chunks').head(10).plot(kind='barh', figsize=(8,3)).invert_yaxis()

## Persisting the Result


In [ ]:
import sqlite3 ###
df[nlp_columns] = df[nlp_columns].applymap(lambda items: ' '.join(items))

con = sqlite3.connect(db_path) 
df.to_sql("posts_nlp", con, index=False, if_exists="replace")
con.close()

### A Note on Execution Time


# There is More
## Language Detection
## Spell Checking
## Token Normalization


# Closing Remarks and Recommendations
